# Description of the problem and solution

The task1 was to predict a person's age from the brain image data: a standard regression problem. The original dataset included 832 features as well as a lot of NaN values and a few outliers. A good preprocessing stage was necessary in order to have a well defined dataset that could be used in our regression model. First step was the imputation of the dataset. Filling each NaN value with the median of each feature column. The use of the median instead of other value (e.g. mean) is justified since a lot of outliers are included in the dataset. (e.g. 1 2 _ 5 20 median: 3 mean: 7). Next step was the feature extraction. By using the "autofeat" library (paper: https://arxiv.org/pdf/1901.07329.pdf), we extracted the 21 most important features. The way the algorithm works is going through a loop of correlation of features with target, select promising features, train Lasso regression model with promising features, filter the good features keeping the ones with non-zero regression weights. We updated the datasets by keeping only the 21 most important features. Finally, we used these updated datasets for the training of our final regression model. A lot of outlier detection techniques were used but we decided to keep the outliers and use a tree-based method for our final model. Tree-methods have been proved to be robust to outliers and we avoid risking excluded important features / points from the dataset. The "ExtraTreesRegressor" model from the "sklearn" package was used and fine tuned based on the R2 score performance in our validation set. The final model had a score >0.6 in the validation sets using cross-validation and in the submission leaderboard of ETH scored 0.6812 while the hard baseline was set to 0.65 by the Advanced Machine Learning Task1 team.

# Include all the necessary packages

In [1]:
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from autofeat import FeatureSelector

from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor

# Load the data from the CSV files

In [2]:
column_names_x = ['id']
for i in range(832):
  column_names_x.append('x'+str(i))

raw_dataset_x = pd.read_csv('Data/X_train.csv', names=column_names_x,
                      na_values = "?", comment='\t',
                      sep=",", skipinitialspace=True, skiprows=1)

dataset_x = raw_dataset_x.copy()
dataset_x.tail()

,id,x0,x1,x2,x3,x4,x5,x6,x7,x8,...,x822,x823,x824,x825,x826,x827,x828,x829,x830,x831
1207,1207.0,18707.457475,13610.725702,3785.886941,113497.632841,109.526764,97.812339,11274.011935,10803.953566,10949.811419,...,8472.132451,17727.003274,1033.057071,104.838553,102.235191,3099.069091,3.079234,1362.889974,NaN,10110.360570
1208,1208.0,17108.239122,12168.536128,3442.619145,98218.773311,100.468476,109.994258,11031.326117,10231.743317,10687.321177,...,8839.251924,14721.087037,792.061138,105.823015,109.915094,3124.059793,2.935047,1577.404870,1026.749434,10620.330033
1209,1209.0,14264.707321,9273.405761,3580.894003,101668.927699,102.620705,104.470375,9159.594864,10661.827392,10623.176915,...,10264.321725,15226.056342,831.025190,101.926717,101.669153,2252.031870,NaN,1781.799720,1066.379647,10317.757445
1210,1210.0,14907.077440,10936.636575,3159.167789,100400.608972,106.622507,84.859872,10356.404262,10107.960852,10384.924460,...,10310.165709,12976.062457,852.001070,103.098317,104.397562,2585.048660,2.731768,1300.379678,1049.370040,10876.010268
1211,1211.0,14975.969273,11451.350347,3107.470343,100080.295868,108.040548,NaN,10766.888640,10726.567333,10632.289329,...,8853.948288,12953.077245,919.051273,107.668141,102.999153,2489.012323,2.721068,1416.819713,1088.315592,10359.138969


In [3]:
column_names_y = ['id','y']
raw_dataset_y = pd.read_csv('Data/y_train.csv', names=column_names_y,
                      na_values = "?", comment='\t',
                      sep=",", skipinitialspace=True, skiprows=1)

dataset_y = raw_dataset_y.copy()
dataset_y.tail()

,id,y
1207,1207.0,70.0
1208,1208.0,86.0
1209,1209.0,68.0
1210,1210.0,71.0
1211,1211.0,53.0


# Print the missing values

In [4]:
missing_values = dataset_x.isna().sum()
print (missing_values)

id        0
x0       91
x1       84
x2       88
x3       91
       ... 
x827    101
x828     95
x829     92
x830     80
x831     96
Length: 833, dtype: int64


# Split the data into training and test data

In [5]:
# Split using sklearn.model_selection
x_train, x_test, y_train, y_test = train_test_split(dataset_x, dataset_y, test_size=0.2, random_state = 100)

In [6]:
train_stats = x_train.describe()
train_stats.pop("id")
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
x0,901.0,15254.390521,2231.563761,6027.219619,13918.413076,15042.836058,16709.756102,28273.690135
x1,902.0,10970.368098,1587.024515,6764.060541,9873.366723,10825.381843,11921.708544,17777.338221
x2,906.0,3434.029571,433.096972,1849.453269,3156.328586,3409.310025,3702.856739,5622.951648
x3,896.0,100121.823708,9950.710491,65828.916291,93444.346839,100160.023492,106380.812492,133145.632257
x4,910.0,105.212591,2.837787,100.067706,102.784971,105.176689,107.681571,110.087261
...,...,...,...,...,...,...,...,...
x827,886.0,2492.950348,513.730065,867.825011,2168.018939,2470.568940,2800.578394,4904.988601
x828,895.0,2.730978,0.264650,1.744389,2.538333,2.715997,2.892752,3.795277
x829,899.0,1359.150032,260.487607,711.119041,1176.638459,1366.071397,1538.372796,2322.003232
x830,908.0,1052.854318,29.229104,1000.067137,1028.688835,1054.615566,1078.409654,1099.975679


# Fill the NaN in the training data set with the median values of each column

In [7]:
x_train = x_train.fillna(x_train.median())
x_test = x_test.fillna(x_test.median())
missing_values = x_train.isna().sum()
print (missing_values)

id      0
x0      0
x1      0
x2      0
x3      0
       ..
x827    0
x828    0
x829    0
x830    0
x831    0
Length: 833, dtype: int64


# Remove the unnecessary "id" label

In [8]:
y_train.pop("id")
y_test.pop("id")

1143    1143.0
941      941.0
365      365.0
467      467.0
615      615.0
         ...  
156      156.0
689      689.0
28        28.0
69        69.0
1198    1198.0
Name: id, Length: 243, dtype: float64

# Feature Extraction using autofeat

In [12]:
fsel = FeatureSelector(featsel_runs=4,
        max_it=150,
        w_thr=1e-6,
        keep=None,
        n_jobs=1,
        verbose=1)

new_X = fsel.fit_transform(pd.DataFrame(x_train, columns=column_names_x), y_train)
print(new_X.columns) 

df_train = pd.DataFrame(x_train, columns=column_names_x)
df_test = pd.DataFrame(x_test, columns=column_names_x)

c:\Users\Hongyi LAN\miniconda3\envs\aml_project\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[featsel] Scaling data...done.
[featsel] 231/833 features after univariate filtering
[featsel] Feature selection run 1/4
[featsel] Feature selection run 2/4
[featsel] Feature selection run 3/4
[featsel] Feature selection run 4/4
[featsel] 37 features after 4 feature selection runs
[featsel] 33 features after correlation filtering
[featsel] 22 features after noise filtering
[featsel] 22 final features selected (including 0 original keep features).
Index(['x68', 'x194', 'x633', 'x92', 'x485', 'x612', 'x641', 'x458', 'x380',
       'x233', 'x465', 'x726', 'x325', 'x89', 'x232', 'x415', 'x209', 'x348',
       'x115', 'x819', 'x742', 'x665'],
      dtype='object')


# Keep only the necessary features

In [10]:
dataset_selected_x = x_train.copy()
x_test_selected= x_test.copy()

#accepted=[400, 757, 635, 516, 132, 15, 809, 116, 214,
       #556, 617, 93, 346, 596, 309, 252, 292, 474,
       #593, 614]

accepted=[400, 635, 757, 516, 809, 214, 556, 617, 93,
       346, 596, 255, 309, 252, 292, 738, 537, 593,
       474, 614, 502]

for j in range(832):
        if (j in accepted):
          print (j)
        else:
          del dataset_selected_x['x'+str(j)]
          del x_test_selected['x'+str(j)]
      
train_stats = dataset_selected_x.describe()
train_stats.pop("id")
train_stats = train_stats.transpose()
train_stats

93
214
252
255
292
309
346
400
474
502
516
537
556
593
596
614
617
635
738
757
809


,count,mean,std,min,25%,50%,75%,max
x93,969.0,4.367681e+03,5.895625e+02,2.162708e+03,4.015083e+03,4.348544e+03,4.679013e+03,7.382366e+03
x214,969.0,2.303067e+00,1.842725e-01,1.526802e+00,2.195638e+00,2.306954e+00,2.412040e+00,2.927466e+00
x252,969.0,1.050452e+04,2.697904e+02,1.000014e+04,1.028982e+04,1.050314e+04,1.072745e+04,1.099966e+04
x255,969.0,1.005422e+02,9.167929e+00,6.226260e+01,9.450646e+01,1.008159e+02,1.060984e+02,1.335456e+02
x292,969.0,3.782105e+04,2.802936e-02,3.782101e+04,3.782103e+04,3.782105e+04,3.782108e+04,3.782111e+04
x309,969.0,2.078555e+00,1.601533e-01,1.357216e+00,1.986457e+00,2.083465e+00,2.170873e+00,2.700711e+00
x346,969.0,1.049018e+06,2.686707e+04,1.000087e+06,1.026117e+06,1.049662e+06,1.068778e+06,1.099860e+06
x400,969.0,1.004372e+03,9.972676e+01,5.827449e+02,9.404754e+02,1.006425e+03,1.066169e+03,1.468679e+03
x474,969.0,1.051360e+06,2.687400e+04,1.000022e+06,1.030836e+06,1.052140e+06,1.072486e+06,1.099845e+06
x502,969.0,1.569304e+04,2.050646e+03,6.837685e+03,1.450654e+04,1.556246e+04,1.695634e+04,2.909847e+04


# Regression model

In [11]:
rfr = ExtraTreesRegressor(n_jobs=1, max_depth=None, n_estimators=180, random_state=0, min_samples_split=3, max_features=None)

rfr.fit(dataset_selected_x, np.ravel(y_train))

y_pred = rfr.predict(x_test_selected) 

score = r2_score(y_test, y_pred)
print(score)

0.06651028255721181


# Export the file with our predictions for submission

In [ ]:
#accepted=[400, 757, 635, 516, 132, 15, 809, 116, 214,
       #556, 617, 93, 346, 596, 309, 252, 292, 474,
       #593, 614]

accepted=[400, 635, 757, 516, 809, 214, 556, 617, 93,
       346, 596, 255, 309, 252, 292, 738, 537, 593,
       474, 614, 502]

column_names_x = ['id']
for i in range(832):
  column_names_x.append('x'+str(i))

raw_dataset_x_test = pd.read_csv('/content/X_test.csv', names=column_names_x,
                      na_values = "?", comment='\t',
                      sep=",", skipinitialspace=True, skiprows=True)

dataset_x_test = raw_dataset_x_test.copy()
dataset_x_test.tail()

dataset_x_test = dataset_x_test.fillna(dataset_x_test.median())

dataset_selected_x_test = dataset_x_test.copy()

for j in range(832):
        if (j in accepted):
          print (j)
        else:
          del dataset_selected_x_test['x'+str(j)]

predictions = rfr.predict(dataset_selected_x_test)

index = 0.0
with open('predictions.txt', 'w') as f:
    f.write("%s\n" % "id,y")
    for predict in predictions:
        writing_str = str(index)+','+str(predict.item(0))
        f.write("%s\n" % writing_str)
        index = index + 1

93
214
252
255
292
309
346
400
474
502
516
537
556
593
596
614
617
635
738
757
809
